In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

#Torch-related imports
import torch
import torch.distributions as D
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Function

#Model-specific imports
from SBM_SDE import *
from obs_and_flow import *
from training import calc_log_lik

In [18]:
torch.manual_seed(0)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

dt_flow = 0.2 #SDE discretization timestep.
t = 250 #Simulation run for T hours.
n = int(t / dt_flow) + 1
t_span = np.linspace(0, t, n)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
niter = 2200
piter = 50
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.
pretrain_lr = 1e-2
train_lr = 1e-3
batch_size = 10 #Number of sets of observation outputs to sample per set of parameters.

In [19]:
temp_ref = 283
temp_rise = 5 #High estimate of 5 celsius temperature rise by 2100. 

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.05

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}

In [20]:
obs_error_scale = 0.1

x0_SCON = [37, 0.1, 0.9]
x0_SCON_tensor = torch.tensor(x0_SCON)
x0_prior_SCON = D.multivariate_normal.MultivariateNormal(x0_SCON_tensor,
                                                         scale_tril=torch.eye(state_dim_SCON) * obs_error_scale * x0_SCON_tensor)

In [21]:
obs_times, obs_means_CON, obs_error_CON = csv_to_obs_df('y_from_x_t_250_dt_0-01.csv', state_dim_SCON, t, obs_error_scale)

In [22]:
#Obtain temperature forcing function.
temp_tensor = temp_gen(t_span_tensor, temp_ref, temp_rise)

#Obtain SOC and DOC pool litter input vectors for use in flow SDE functions.
i_s_tensor = i_s(t_span_tensor) #Exogenous SOC input function
i_d_tensor = i_d(t_span_tensor) #Exogenous DOC input function

In [23]:
obs_model_CON_noCO2 = ObsModel(DEVICE = devi, TIMES = obs_times, DT = dt_flow, MU = obs_means_CON, SCALE = obs_error_CON)

In [24]:
def calc_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, TEMP_TENSOR, TEMP_REF, DRIFT_DIFFUSION, X0_PRIOR, PARAMS_DICT):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], TEMP_TENSOR[:, :-1, :], TEMP_REF, PARAMS_DICT)
    euler_maruyama_state_sample_object = D.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    
    # Compute log p(x|theta) = log p(x|x0, theta) + log p(x0|theta)
    ll = euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1) # log p(x|x0, theta)
    ll += X0_PRIOR.log_prob(C_PATH[:, 0, :]) # log p(x0|theta)
    
    return ll # (batch_size, )

In [25]:
def train(DEVICE, PRETRAIN_LR, TRAIN_LR, NITER, PRETRAIN_ITER, BATCH_SIZE, OBS_MODEL,
          STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, TEMP_TENSOR, TEMP_REF,
          DRIFT_DIFFUSION, X0_PRIOR, PARAMS_DICT,
          LEARN_PARAMS = False, LR_DECAY = 0.1, DECAY_STEP_SIZE = 1000, PRINT_EVERY = 500):
    net = SDEFlow(DEVICE, OBS_MODEL, STATE_DIM, T, DT, N, num_layers = 7).to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr = PRETRAIN_LR)
    
    if LEARN_PARAMS:
        theta_post = MeanField(PARAMS_DICT)
        theta_prior = D.normal.Normal(torch.zeros_like(theta_post.means),
                                      torch.ones_like(theta_post.std))
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e10
    norm_losses = [] #[best_loss_norm] * 10 
    ELBO_losses = [] #[best_loss_ELBO] * 10
    #C0 = ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), PARAMS_DICT) #Calculate deterministic initial conditions.
    #C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Assign initial conditions to C_PATH.
    
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for it in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net(BATCH_SIZE) #Obtain paths with solutions at times after t0.
            #C_PATH = torch.cat([C0, C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            
            if it < PRETRAIN_ITER:
                l1_norm_element = C_PATH - torch.mean(OBS_MODEL.mu[:3], -1)
                l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l1_norm.item())
                #l2_norm_element = C_PATH - torch.mean(OBS_MODEL.mu, -1)
                #l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                #best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                #norm_losses.append(l2_norm.item())
                
                if (it + 1) % PRINT_EVERY == 0:
                    print(f"Moving average norm loss at {it + 1} iterations is: {sum(norm_losses[-10:]) / len(norm_losses[-10:])}. Best norm loss value is: {best_loss_norm}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
                l1_norm.backward()
                #l2_norm.backward()
                
            else:
                if LEARN_PARAMS:
                    theta_dict, theta, log_q_theta = theta_post()
                    log_p_theta = theta_prior.log_prob(theta).sum(-1)
                else:
                    theta_dict = PARAMS_DICT
                    log_q_theta, log_p_theta = torch.zeros(2)
                log_lik = calc_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), DT, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE),
                                       TEMP_TENSOR, TEMP_REF, DRIFT_DIFFUSION, X0_PRIOR, theta_dict)
                
                # - log p(theta) + log q(theta) + log q(x|theta) - log p(x|theta) - log p(y|x, theta)
                ELBO = -log_p_theta.mean() + log_q_theta.mean() - log_lik.mean() - OBS_MODEL(C_PATH, theta_dict) + log_prob.mean()
                best_loss_ELBO = ELBO if ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(ELBO.item())

                if (it + 1) % PRINT_EVERY == 0:
                    print(f"Moving average ELBO loss at {it + 1} iterations is: {sum(ELBO_losses[-10:]) / len(ELBO_losses[-10:])}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\n C_PATH =', C_PATH)
                    print(theta_dict)
                ELBO.backward()
                
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            if it == PRETRAIN_ITER:
                optimizer.param_groups[0]['lr'] = TRAIN_LR
            elif it % DECAY_STEP_SIZE == 0 and it > PRETRAIN_ITER:
                optimizer.param_groups[0]['lr'] *= LR_DECAY
            optimizer.step()
            tq.update()
            
    return net, ELBO_losses

In [26]:
net_batch_10_dt_flow_0_2, elbo_hist_batch_10_dt_flow_0_2 = train(devi, pretrain_lr, train_lr, niter, piter, batch_size, obs_model_CON_noCO2,
                       state_dim_SCON, t, dt_flow, n, t_span_tensor, i_s_tensor, i_d_tensor, temp_tensor, temp_ref,
                       drift_diffusion_SCON_C, x0_prior_SCON, SCON_C_params_dict,
                       LR_DECAY = 0.1, DECAY_STEP_SIZE = 5000, PRINT_EVERY = 20)


Train Diffusion:   1%|          | 19/2200 [02:05<4:07:02,  6.80s/it]

Moving average norm loss at 20 iterations is: 381897.159375. Best norm loss value is: 379952.25.

C_PATH mean = tensor([[1.7509, 1.7300, 1.7481],
        [1.7526, 1.7299, 1.7384],
        [1.7506, 1.7317, 1.7354],
        [1.7515, 1.7240, 1.7422],
        [1.7556, 1.7319, 1.7437],
        [1.7487, 1.7273, 1.7426],
        [1.7550, 1.7302, 1.7380],
        [1.7465, 1.7366, 1.7461],
        [1.7515, 1.7347, 1.7477],
        [1.7488, 1.7269, 1.7395]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[1.2846, 2.0741, 1.7830],
         [1.5008, 1.5309, 1.5655],
         [1.5080, 1.4883, 1.4751],
         ...,
         [1.7999, 2.0723, 1.9963],
         [1.8099, 1.9339, 1.8996],
         [1.9000, 1.9868, 1.8119]],

        [[2.2386, 1.5710, 1.6057],
         [1.5871, 1.5022, 1.5390],
         [1.5120, 1.5168, 1.4998],
         ...,
         [2.1553, 2.1221, 1.9819],
         [1.6007, 1.8264, 2.1494],
         [1.8871, 1.9030, 2.1736]],

        [[1.2243, 1.5896, 1.5379],
         [1.5409, 1.5224,


Train Diffusion:   2%|▏         | 39/2200 [04:13<3:40:09,  6.11s/it]

Moving average norm loss at 40 iterations is: 367635.809375. Best norm loss value is: 335138.8125.

C_PATH mean = tensor([[6.3320, 1.3909, 2.1964],
        [6.4895, 1.2605, 2.1353],
        [6.6678, 1.3695, 2.1205],
        [6.6565, 1.3231, 2.1691],
        [6.7758, 1.1778, 2.1829],
        [6.6315, 1.3833, 2.2206],
        [6.1893, 1.2581, 2.1604],
        [6.6728, 1.2616, 2.1515],
        [6.3876, 1.2707, 2.1179],
        [6.4209, 1.2913, 2.2077]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[3.4397e+00, 9.6461e-01, 7.9647e-01],
         [1.3439e+01, 3.0523e-01, 1.1216e+01],
         [2.7091e+00, 8.2507e-01, 1.2765e+00],
         ...,
         [6.6265e-01, 8.3804e-01, 1.0045e+00],
         [9.9052e-01, 5.4575e+00, 2.9963e-01],
         [3.0741e-01, 1.8066e+00, 3.4728e+00]],

        [[4.1695e+00, 5.6732e-01, 9.8999e-01],
         [2.8062e+01, 1.3180e+00, 6.7675e-01],
         [1.6098e+00, 8.3818e-01, 9.3220e-01],
         ...,
         [1.6359e+01, 2.3367e-01, 9.1332e-01],
         [


Train Diffusion:   3%|▎         | 59/2200 [06:20<3:49:25,  6.43s/it]

...

Moving average ELBO loss at 1100 iterations is: 4301.49404296875. Best ELBO loss value is: 4270.083984375.

C_PATH mean = tensor([[1.0947, 0.7052, 0.9360],
        [1.0885, 0.7134, 0.9307],
        [1.0798, 0.7124, 0.9346],
        [1.0785, 0.7133, 0.9319],
        [1.0310, 0.7141, 0.9320],
        [1.0890, 0.7092, 0.9328],
        [1.0940, 0.7057, 0.9418],
        [1.1239, 0.7065, 0.9335],
        [1.0560, 0.7087, 0.9240],
        [1.0889, 0.7130, 0.9398]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[2.0304, 0.1141, 0.7947],
         [0.9177, 0.1531, 0.4808],
         [1.6089, 0.1732, 0.3737],
         ...,
         [1.7124, 0.7848, 1.2318],
         [0.9063, 0.8327, 1.2900],
         [1.3916, 0.8354, 1.2667]],

        [[0.7000, 0.1054, 0.7724],
         [0.7712, 0.2003, 0.4483],
         [0.6691, 0.2978, 0.5712],
         ...,
         [0.7593, 0.7682, 1.1169],
         [1.5475, 0.7159, 1.1815],
         [1.3073, 0.7804, 1.3099]],

        [[0.0946, 0.1092, 0.6930],
         [0.35


Train Diffusion:  51%|█████     | 1119/2200 [2:00:17<2:02:09,  6.78s/it]

Moving average ELBO loss at 1120 iterations is: 4251.472900390625. Best ELBO loss value is: 4233.5185546875.

C_PATH mean = tensor([[1.0794, 0.7117, 0.9538],
        [1.0862, 0.7088, 0.9552],
        [1.0782, 0.7108, 0.9415],
        [1.0875, 0.7099, 0.9455],
        [1.1083, 0.7116, 0.9497],
        [1.0879, 0.7105, 0.9395],
        [1.0334, 0.7088, 0.9314],
        [1.0736, 0.7064, 0.9569],
        [1.0638, 0.7106, 0.9475],
        [1.0998, 0.7058, 0.9605]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[0.1335, 0.1135, 0.7164],
         [0.5901, 0.2100, 0.5261],
         [0.4792, 0.2892, 0.4490],
         ...,
         [0.9551, 0.7563, 0.9356],
         [1.0160, 0.6950, 0.8902],
         [1.3084, 0.6812, 1.2444]],

        [[0.5019, 0.1038, 0.6965],
         [0.5575, 0.1941, 0.5207],
         [1.0465, 0.2554, 0.4423],
         ...,
         [1.0534, 0.7000, 0.9728],
         [1.4879, 0.7080, 1.2150],
         [1.8294, 0.7149, 1.5524]],

        [[0.6391, 0.0946, 0.8333],
         [0.


Train Diffusion:  51%|█████     | 1121/2200 [2:00:37<1:56:06,  6.46s/it]


KeyboardInterrupt: 

In [ ]:
torch.save(net, f'net_t_{t}_dt_{dt_flow}.pt')

In [ ]:
def plot_post(x, obs_model, state_idx=0, num_samples=20,
              ymin=None, ymax=None):
    #net.eval()
    #x, _ = net(num_samples)
    #x0 = x0[(None,) * 2].repeat(num_samples, 1, 1)
    #x = torch.cat((x0, x), 1)
    
    q_mean, q_std = x[:, :, state_idx].mean(0).detach(), x[:, :, state_idx].std(0).detach()
    hours = torch.arange(0, t + dt, dt)
    plt.plot(hours, q_mean, label='Posterior mean')
    plt.fill_between(hours, q_mean - 2*q_std, q_mean + 2*q_std, alpha=0.5,
                     label='Posterior $\\mu \pm 2\sigma$')
    plt.plot(obs_model.times, obs_model.mu[state_idx, :], linestyle='None', marker='o',
             label='Observed')
    
    plt.legend()
    plt.xlabel('Hour')
    plt.ylabel(['SOC', 'DOC', 'MBC'][state_idx])
    plt.ylim((ymin, ymax))
    plt.title('Approximate posterior $q(x|\\theta, y)$\nNumber of samples = {}'.format(num_samples))